In [11]:
# In a Jupyter Notebook cell
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# --- Functions ---

def read_fchk(file_path):
    """
    Extract atomic numbers and Cartesian coordinates from a Gaussian .fchk file.
    
    Args:
        file_path (str): Path to the .fchk file.
    
    Returns:
        np.ndarray: Array of shape (n, 4) with [x, y, z, atomic_number] for each atom.
    """
    atomic_numbers = []
    coordinates = []
    num_atoms = 0
    
    with open(file_path, 'r') as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            if line.startswith('Number of atoms'):
                num_atoms = int(line.split()[-1])
            elif line.startswith('Atomic numbers'):
                start_line = i + 1
                values = []
                while len(values) < num_atoms and start_line < len(lines):
                    values.extend([int(x) for x in lines[start_line].split()])
                    start_line += 1
                atomic_numbers = values[:num_atoms]
            elif line.startswith('Current cartesian coordinates'):
                start_line = i + 1
                values = []
                while len(values) < 3 * num_atoms and start_line < len(lines):
                    values.extend([float(x) for x in lines[start_line].split()])
                    start_line += 1
                coordinates = values[:3 * num_atoms]
    
    if not atomic_numbers or not coordinates or num_atoms == 0:
        raise ValueError("Failed to parse atomic numbers or coordinates from .fchk file.")
    
    coordinates = np.array(coordinates).reshape(num_atoms, 3)
    atomic_numbers = np.array(atomic_numbers)
    
    return coordinates, atomic_numbers


def rotate(coords, phi, theta, psi):
    """
    Rotate coordinates by Euler angles (Z(phi) Y(theta) Z(psi)).
    Angles in radians.
    """
    Rz1 = np.array([
        [ np.cos(phi), -np.sin(phi), 0],
        [ np.sin(phi),  np.cos(phi), 0],
        [           0,            0, 1]
    ])
    Ry = np.array([
        [ np.cos(theta), 0, np.sin(theta)],
        [            0, 1,            0],
        [-np.sin(theta), 0, np.cos(theta)]
    ])
    Rz2 = np.array([
        [ np.cos(psi), -np.sin(psi), 0],
        [ np.sin(psi),  np.cos(psi), 0],
        [           0,            0, 1]
    ])
    R = Rz1 @ Ry @ Rz2
    return coords.dot(R.T)

# Color/size maps for plotting
atom_colors = {1:'white', 6:'black', 7:'blue', 8:'red', 16:'yellow'}
atom_sizes = {1:50, 6:100, 7:100, 8:100, 16:150}


def plot_molecule(numbers, coords, ax, title=None):
    for num, (x,y,z) in zip(numbers, coords):
        ax.scatter(x, y, z,
                   color=atom_colors.get(num, 'gray'),
                   s=atom_sizes.get(num, 80))
    if title:
        ax.set_title(title)
    ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')
    # Equal aspect ratio
    max_range = np.ptp(coords, axis=0).max()
    mid = coords.mean(axis=0)
    for i, axis_name in enumerate(['X','Y','Z']):
        low, high = mid[i]-max_range/2, mid[i]+max_range/2
        getattr(ax, f'set_{axis_name.lower()}lim')((low, high))

# --- Usage Example ---
# 1) Read your .fchk file
numbers, coords = read_fchk('/scratch/phys/sin/sethih1/data_files/first_group/7492.fchk')

# 2) Set Euler angles (degrees) and convert
phi_deg, theta_deg, psi_deg = 30, 45, 60
phi, theta, psi = np.deg2rad([phi_deg, theta_deg, psi_deg])

# 3) Rotate coordinates
coords_rot = rotate(coords, phi, theta, psi)

# 4) Plot original and rotated side by side
fig = plt.figure(figsize=(12,6))
ax1 = fig.add_subplot(121, projection='3d')
plot_molecule(numbers, coords, ax1, title='Original')
ax2 = fig.add_subplot(122, projection='3d')
plot_molecule(numbers, coords_rot, ax2,
              title=f'Rotated φ={phi_deg}°, θ={theta_deg}°, ψ={psi_deg}°')
plt.tight_layout()
plt.show()


ValueError: shapes (16,) and (3,3) not aligned: 16 (dim 0) != 3 (dim 0)

In [ ]:
/scratch/phys/sin/sethih1/data_files/first_group/7492.fchk

In [15]:
# In a Jupyter Notebook cell
import numpy as np
from ase import Atoms
import nglview as nv

# --- Functions ---

def read_fchk(filename):
    """
    Parse a Gaussian .fchk file.
    Returns:
      numbers: (N,) array of atomic numbers
      coords:  (N,3) array of Cartesian coordinates (Å)
    """
    with open(filename, 'r') as f:
        lines = f.readlines()

    # Number of atoms
    natoms = None
    for line in lines:
        if 'Number of atoms' in line:
            natoms = int(line.split()[-1])
            break
    if natoms is None:
        raise ValueError("Could not find number of atoms")

    # Atomic numbers
    numbers = []
    for i, line in enumerate(lines):
        if 'Atomic numbers' in line:
            count = int(line.split('N=')[1].split()[0]) if 'N=' in line else int(lines[i+1].split()[0])
            j = i + 1
            while len(numbers) < count:
                numbers.extend(map(int, lines[j].split()))
                j += 1
            break
    if len(numbers) != natoms:
        raise ValueError(f"Atomic numbers count mismatch: expected {natoms}, got {len(numbers)}")

    # Coordinates
    coords = []
    for i, line in enumerate(lines):
        if 'Current cartesian coordinates' in line or 'Cartesian Coordinates' in line:
            count = int(line.split('N=')[1].split()[0]) if 'N=' in line else int(lines[i+1].split()[0])
            j = i + 1
            while len(coords) < count:
                coords.extend(map(float, lines[j].split()))
                j += 1
            coords = np.array(coords[:count]).reshape(-1,3)
            break
    if coords.shape[0] != natoms:
        raise ValueError(f"Coordinate count mismatch: expected {natoms}, got {coords.shape[0]}")

    return np.array(numbers), coords


def rotate(coords, phi, theta, psi):
    """
    Rotate coords by Euler Z(phi)-Y(theta)-Z(psi) in radians
    """
    Rz1 = np.array([[np.cos(phi), -np.sin(phi), 0],
                    [np.sin(phi),  np.cos(phi), 0],
                    [0,            0,           1]])
    Ry  = np.array([[ np.cos(theta), 0, np.sin(theta)],
                    [ 0,             1, 0           ],
                    [-np.sin(theta), 0, np.cos(theta)]])
    Rz2 = np.array([[np.cos(psi), -np.sin(psi), 0],
                    [np.sin(psi),  np.cos(psi), 0],
                    [0,            0,           1]])
    return coords.dot((Rz1 @ Ry @ Rz2).T)


def show_nglview(numbers, coords, viewer_kwargs=None, component_kwargs=None):
    """
    Render molecule in Jupyter via nglview.
    numbers: array of atomic numbers
    coords:   array of shape (N,3)
    viewer_kwargs: dict passed to nv.show_ase viewer
    component_kwargs: dict of kwargs for add_ball_and_stick
    """
    # Map atomic numbers to symbols
    symbols_map = {1:'H', 6:'C', 7:'N', 8:'O', 16:'S'}
    symbols = [symbols_map.get(z, 'X') for z in numbers]

    # Create ASE Atoms object
    mol = Atoms(symbols=symbols, positions=coords)

    # Initialize nglview viewer
    view = nv.show_ase(mol, **(viewer_kwargs or {}))
    # Add ball-and-stick representation by default
    view.add_ball_and_stick(**(component_kwargs or {}))
    view.center()
    return view

# --- Example Usage in Notebook ---
# 1) Read your .fchk file
numbers, coords = read_fchk('/scratch/phys/sin/sethih1/data_files/first_group/7492.fchk')

# 2) Show original molecule in nglview
view_orig = show_nglview(numbers, coords)
view_orig

# 3) Rotate by Euler angles (degrees)
phi, theta, psi = np.deg2rad([30, 45, 60])
coords_rot = rotate(coords, phi, theta, psi)

# 4) Show rotated molecule
view_rot = show_nglview(numbers, coords_rot)
view_rot

ModuleNotFoundError: No module named 'nglview'